# Chance Performance

## Random age picks

In [5]:
def random_its(num_its, df_random_ages, subject_ages):
    import random
    import pandas as pd
    
    ages = {}
    # generate 1000 iterations of random ages
    for i in range(num_its):
        rand_age = []
        for sub in range(len(df_random_ages)):
            rand_indx = random.randrange(0, len(subject_ages))
            rand_age.append(subject_ages[rand_indx])
        ages[str(i+1)] = rand_age
        #df_temp = pd.DataFrame(rand_age)
        #df_temp.columns = [str(i+1)]
        #df_random_ages = pd.concat([df_random_ages, df_temp], axis=1)
        
        if(i%100 == 0):
            print(i)
    
    df_random_ages = pd.DataFrame(ages)
    
    return df_random_ages

## MAE of random ages

In [4]:
def get_arr_mae(df_random_ages, subject_ages):
    # calculate MAE for each random prediction
    # test mean absolute error
    from sklearn.metrics import mean_absolute_error, r2_score

    arr_mae = []
    for i in range(1, df_random_ages.shape[1]):
        arr_mae.append(mean_absolute_error(subject_ages, df_random_ages[str(i)]))
    
    return arr_mae

## Get random ages and MAE

In [3]:
def random_ages(df):
    import pandas as pd
    # get all the subject ages
    print("Picking Random Ages")
    random_ages = pd.DataFrame(df.subject.values, columns = [['subject']])
    random_ages = random_its(1000, random_ages, df.age)
    
    print("Calculating Mean Absolute Errors")
    mae = get_arr_mae(random_ages, df.age)
    
    return mae

## Plot random age historgram

In [3]:
def plot_rand_hist(mae, lines=None, xlim_min=0, x_max = None, title="", set_context='notebook', fig_tuple=(5,5), legend=True):
    
#     if not x_max:
#         x_max = max(mae)
    
    # plot MAE Distribution
    import matplotlib.pyplot as plt
    import numpy as np
    import matplotlib.mlab as mlab
    import seaborn as sns
    
    sns.set_context(set_context, font_scale=1.2)
    # plotting the Histogram
    plt.figure(1)
    plt.figure(figsize=fig_tuple)
    plt.hist(mae, normed = True, label='Random MAE Distribution')
    plt.xlim(xlim_min, x_max)
    plt.xlabel("Mean Absolute Error")
    #plt.ylabel("Percentage of mean absolute error in each group")

    colors = ['darkblue', 'slateblue', 'rebeccapurple', 'darkviolet', 'violet', 'fuchsia', 'deeppink', 'crimson']
    color_counter = 0
    
    if lines:
        for key, x in lines.items():
            # plot MAE complex
            plt.axvline(x, ymax=1, label=key, color=colors[color_counter])
            color_counter+=1
    
    plt.title(title)
    
    if legend:
        plt.legend(loc='best')
        
    plt.show()

## Percentile

In [15]:
def percentile(mae, x):
    # Calculating Percentile of Score of Complex Model
    from scipy.stats import percentileofscore

    percentile_mae = percentileofscore(mae, x)
    return percentile_mae

In [14]:
def get_percentile(mae, lines, data=''):
    for title, x in lines.items():
        print("Percentile of " + data + " for " + title, percentile(mae,x))

## Games Howell

In [2]:
def games_howell(group1, group2):
    import math
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    from statsmodels.stats.multicomp import MultiComparison
    import rpy2 as rpy2
    import rpy2.robjects as ro

    mean_diff = abs(np.mean(group1) - np.mean(group2))
    s1 = np.var(group1)
    s2 = np.var(group2)
    
    ss_sums = ((s1**2)/(len(group1)))+((s2**2)/(len(group2)))
    sigma = math.sqrt(0.5*ss_sums)
    v = (ss_sums**2)/((((s1**2/len(group1))**2)/(len(group1)-1))+(((s2**2/len(group2))**2)/(len(group2)-1)))
    t = mean_diff/math.sqrt(ss_sums)
    
    ro.r.assign('rt', t)
    ro.r.assign('groups', 2)
    ro.r.assign('df', v)
    p = ro.r('p <- ptukey(rt * sqrt(2), groups, df, lower.tail = FALSE)')
    #ro.r('print(p)')
    return p